# 시작

In [1]:
import xarray as xr
import numpy as np
import his_utils, his_plot
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from multiprocessing import Pool

In [2]:
# 서로 다른 dataset별로 어떻게 생겨먹었는지 다 plot해서 뽑아버리기

target_var_list = ['total_precipitation_6hr', 
                   'mean_sea_level_pressure', 
                   '2m_temperature']

max_time_step = 10

ERA5 = xr.open_dataset("testdata/2022-01-01/ERA5_2022-01-01.nc")
google = xr.open_dataset('testdata/2022-01-01/source-era5_date-2022-01-01_res-0.25_levels-37_steps-12.nc')
    
ERA5 = his_utils.convert_scale(ERA5)
google = his_utils.convert_scale(google)

for var in target_var_list:
    if 'level' in ERA5[var].dims:
        diff = (ERA5[var] - google[var]).mean(dim='level')
    else:
        diff = ERA5[var] - google[var]

    weights = np.cos(np.deg2rad(diff.lat))
    weights.name = "weights"
    weighted = diff.weighted(weights)
    std_val = weighted.std(('lat', 'lon')).max()
    
    norm = TwoSlopeNorm(vmin=-1.5*std_val, vcenter=0, vmax=1.5*std_val)

    arg_list = [(diff.isel(time=time_index, lat=slice(0, 90), lon=slice(20, 150)), 
                 var, 
                 "platecarree", 
                 "RdBu_r", 
                 f'{var} difference at t:{time_index}', 
                 f'figure/diff {var}_{time_index}.png',
                 norm)
        for time_index in range(max_time_step)]

    with Pool() as pool:
        pool.map(his_plot.plot, arg_list)

figure/diff total_precipitation_6hr_1.png saved
figure/diff total_precipitation_6hr_0.png saved
figure/diff total_precipitation_6hr_8.png saved
figure/diff total_precipitation_6hr_9.png savedfigure/diff total_precipitation_6hr_2.png saved

figure/diff total_precipitation_6hr_4.png saved
figure/diff total_precipitation_6hr_5.png saved
figure/diff total_precipitation_6hr_7.png savedfigure/diff total_precipitation_6hr_6.png saved

figure/diff total_precipitation_6hr_3.png saved
figure/diff mean_sea_level_pressure_0.png saved
figure/diff mean_sea_level_pressure_7.png saved
figure/diff mean_sea_level_pressure_5.png saved
figure/diff mean_sea_level_pressure_8.png saved
figure/diff mean_sea_level_pressure_9.png saved
figure/diff mean_sea_level_pressure_2.png saved
figure/diff mean_sea_level_pressure_6.png saved
figure/diff mean_sea_level_pressure_4.png savedfigure/diff mean_sea_level_pressure_1.png saved

figure/diff mean_sea_level_pressure_3.png saved
figure/diff 2m_temperature_1.png saved
f

In [ ]:
# calculate weighted means & max value of each variables
ERA5 = xr.open_dataset("testdata/2022-01-01_tp.grib")
ERA5 = ERA5.stack(new_time=['time', 'step'])
ERA5 = ERA5.assign_coords(new_time=ERA5.valid_time.values)
ERA5 = ERA5.rename({'new_time': 'time'})

ERA5 = ERA5.drop_vars(['number', 'surface'])
tp = "tp"

weights = np.cos(np.deg2rad(ERA5.latitude))
weights.name = "weights"

mean_ERA = {}
mean_google = {}

max_ERA = {}
max_google = {}

for t in range(0, 14):
    mean_ERA[t]=ERA5[tp].isel(time=t).weighted(weights).mean(('latitude', 'longitude')).values * 1000
    # mean_google[t]=ERA5[tp].isel(time=t).weighted(weights).mean(('lat', 'lon')).values

    max_ERA[t]=ERA5[tp].isel(time=t).max(('latitude', 'longitude')).values * 1000
    # max_google[t]=ERA5[tp].isel(time=t).max(('lat', 'lon')).values

# for var in ERA5.data_vars:
#     if var not in ['land_sea_mask', 'geopotential_at_surface']:
#         if 'level' in ERA5[var].dims:
#             mean_ERA[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).mean(dim='level').values
#             mean_google[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).mean(dim='level').values
#         else:
#             mean_ERA[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
#             mean_google[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
        
#     else:
#         mean_ERA[var]=ERA5[var].weighted(weights).mean(('lat', 'lon')).values
#         mean_google[var]=ERA5[var].weighted(weights).mean(('lat', 'lon')).values

In [ ]:
mean_ERA - mean_google